In [1]:
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 7.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.3 which is incompatible.


In [2]:
import cv2
import mediapipe as mp
import numpy as np
import joblib
from sklearn.ensemble import RandomForestClassifier


In [3]:
def extract_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)

    cap.release()
    return frames


In [4]:
def detect_keypoints(frame):
    mp_pose = mp.solutions.pose
    with mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5) as pose:
        results = pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        keypoints = []
        if results.pose_landmarks:
            for landmark in results.pose_landmarks.landmark:
                keypoints.append({
                    'x': landmark.x,
                    'y': landmark.y,
                    'z': landmark.z if hasattr(landmark, 'z') else None,
                    'visibility': landmark.visibility,
                })
        return keypoints


In [5]:
def calculate_distances(keypoints):
    distances = {
        'Distance Left Shoulder-Elbow': np.linalg.norm(np.array([keypoints[11]['x'], keypoints[11]['y']]) - np.array([keypoints[13]['x'], keypoints[13]['y']])),
        'Distance Left Elbow-Wrist': np.linalg.norm(np.array([keypoints[13]['x'], keypoints[13]['y']]) - np.array([keypoints[15]['x'], keypoints[15]['y']])),
        'Distance Right Shoulder-Elbow': np.linalg.norm(np.array([keypoints[12]['x'], keypoints[12]['y']]) - np.array([keypoints[14]['x'], keypoints[14]['y']])),
        'Distance Right Elbow-Wrist': np.linalg.norm(np.array([keypoints[14]['x'], keypoints[14]['y']]) - np.array([keypoints[16]['x'], keypoints[16]['y']])),
        'Distance Left Hip-Knee': np.linalg.norm(np.array([keypoints[23]['x'], keypoints[23]['y']]) - np.array([keypoints[25]['x'], keypoints[25]['y']])),
        'Distance Left Knee-Ankle': np.linalg.norm(np.array([keypoints[25]['x'], keypoints[25]['y']]) - np.array([keypoints[27]['x'], keypoints[27]['y']])),
        'Distance Right Hip-Knee': np.linalg.norm(np.array([keypoints[24]['x'], keypoints[24]['y']]) - np.array([keypoints[26]['x'], keypoints[26]['y']])),
        'Distance Right Knee-Ankle': np.linalg.norm(np.array([keypoints[26]['x'], keypoints[26]['y']]) - np.array([keypoints[28]['x'], keypoints[28]['y']])),
    }
    return distances


In [6]:
def calculate_angle(point1, point2, point3):
    angle_rad = np.arctan2(point3['y'] - point2['y'], point3['x'] - point2['x']) - np.arctan2(point1['y'] - point2['y'], point1['x'] - point2['x'])
    angle_deg = np.abs(np.degrees(angle_rad))
    return angle_deg

def calculate_angles(keypoints):
    angles = {
        'Angle Left Shoulder-Elbow-Wrist': calculate_angle(keypoints[11], keypoints[13], keypoints[15]),
        'Angle Right Shoulder-Elbow-Wrist': calculate_angle(keypoints[12], keypoints[14], keypoints[16]),
        'Angle Left Knee-Hip-Ankle': calculate_angle(keypoints[25], keypoints[23], keypoints[27]),
        'Angle Left Hip-Knee-Ankle': calculate_angle(keypoints[23], keypoints[25], keypoints[27]),
    }
    return angles


In [7]:
def process_video(video_path):
    frames = extract_frames(video_path)
    data = []

    for frame in frames:
        keypoints = detect_keypoints(frame)
        if keypoints:
            distances = calculate_distances(keypoints)
            angles = calculate_angles(keypoints)
            data.append({
                'Frame': frame,
                'Keypoints': keypoints,
                'Distances': distances,
                'Angles': angles,
            })

    return data


In [8]:
def predict_on_frames(frames_data, model):
    predictions = []
    for frame_data in frames_data:
        distances = frame_data['Distances']
        angles = frame_data['Angles']

        features = np.array([distances[key] for key in sorted(distances.keys())] +
                            [angles[key] for key in sorted(angles.keys())]).reshape(1, -1)

        prediction = model.predict(features)
        predictions.append(prediction[0])

    return predictions


In [29]:
def visualize_and_save_predictions(frames_data, predictions, output_video_path):
    height, width, _ = frames_data[0]['Frame'].shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, 20.0, (width, height))

    for frame_data, prediction in zip(frames_data, predictions):
        frame = frame_data['Frame']
        keypoints = frame_data['Keypoints']

        for i, kp in enumerate(keypoints):
            x = int(kp['x'] * frame.shape[1])
            y = int(kp['y'] * frame.shape[0])
            cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)
            cv2.putText(frame, f"({x}, {y})", (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

        connections = mp.solutions.pose.POSE_CONNECTIONS
        for start, end in connections:
            if keypoints[start] and keypoints[end]:
                start_point = (int(keypoints[start]['x'] * frame.shape[1]), int(keypoints[start]['y'] * frame.shape[0]))
                end_point = (int(keypoints[end]['x'] * frame.shape[1]), int(keypoints[end]['y'] * frame.shape[0]))
                cv2.line(frame, start_point, end_point, (0, 255, 255), 2)

        cv2.putText(frame, f"Predicted Exercise: {prediction}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        out.write(frame)

    out.release()


In [30]:
def main(video_path, output_video_path, model_path):
    model = joblib.load(model_path)
    frames_data = process_video(video_path)
    predictions = predict_on_frames(frames_data, model)
    visualize_and_save_predictions(frames_data, predictions, output_video_path)

# Example usage
video_path = '/content/input_video.mp4'
output_video_path = 'output_video.mp4'
model_path = '/content/model.pkl'

main(video_path, output_video_path, model_path)


/usr/local/lib/python3.10/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClas